In [1]:
import os, glob, re, io, random, gensim
import numpy as np

from PyPDF2 import PdfReader
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from nltk.tokenize import word_tokenize


In [2]:
pdfReaders = []
pdfFiles = []
taggedData = []
docLabels = []
wordSets = []

rootDir = "/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored"
txtExtractDir = "/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions/"

In [3]:
# read files and extract to .txt files
def read_files():
    os.chdir(rootDir)
    for file in glob.glob("**/*.pdf"):
        pdfFiles.append(file)
        pdfReaders.append(PdfReader(file))
    print("pdf files read")
        
def extract_to_txt():
    os.chdir(txtExtractDir)
    for i in pdfReaders:
        with open(i.metadata.title + ".txt", 'w', encoding="utf-8") as file:
            
            # add doc title to array for tagging
            docLabels.append(i.metadata.title)
            print("doc labels: " + str(len(docLabels)))
            
            for j in range(0, len(i.pages)):
                # create a text file for future reference / future use
                file.write(i.getPage(j).extract_text())                   
                
    print("text extraction complete")
    
    
def clean_tag_tokenize():
    
    # read from the generated .txt files, clean and append to array
    for i in range (0, len(docLabels)):
        words = open(docLabels[i] + '.txt').read()
        words = words.replace("\n", " ")
        words = words.replace("  ", " ")
        words = words.lower()
        words = ''.join([i for i in words if i.isalpha() or i.isspace() or (i in '.!?:"')])
        words = words.replace(".", " . ")
        words = words.replace("!", " ! ")
        words = words.replace("?", " ? ")
        words = words.replace(":", " : ")
        words = words.replace('"', ' " ')
        words = words.split()
        
        wordSets.append(words)
        print("word sets: " + str(len(wordSets)))
        
    # tag the cleaned words with txt document heading (derived from pdf doc title)
    for i in range (0, len(docLabels)):
        taggedData.append(TaggedDocument(words=wordSets[i], tags=docLabels[i]))
        print(taggedData[0])


In [4]:
read_files()

pdf files read


In [5]:
extract_to_txt()

doc labels: 1
doc labels: 2
text extraction complete


In [6]:
clean_tag_tokenize()

word sets: 1
word sets: 2
TaggedDocument<['new', 'dark', 'agethis', 'ebook', 'is', 'licensed', 'to', 'lena', 'gieseke', 'lenagiesekegmail', '.', 'com', 'on', 'new', 'dark', 'agetechnologyand', 'the', 'end', 'of', 'the', 'futurejames', 'bridle', 'this', 'ebook', 'is', 'licensed', 'to', 'lena', 'gieseke', 'lenagiesekegmail', '.', 'com', 'on', 'this', 'paperback', 'edition', 'first', 'published', 'by', 'verso', 'first', 'published', 'by', 'verso', 'james', 'bridle', 'all', 'rights', 'reservedthe', 'moral', 'rights', 'of', 'the', 'author', 'have', 'been', 'asserted', 'versouk', ':', 'meard', 'street', 'london', 'wf', 'egus', ':', 'jay', 'street', 'suite', 'brooklyn', 'ny', 'versobooks', '.', 'comverso', 'is', 'the', 'imprint', 'of', 'new', 'left', 'booksisbn', ':', 'isbn', ':', 'uk', 'ebkisbn', ':', 'us', 'ebkbritish', 'library', 'cataloguing', 'in', 'publication', 'dataa', 'catalogue', 'record', 'for', 'this', 'book', 'isavailable', 'from', 'the', 'british', 'librarylibrary', 'of', 'congr

In [8]:
# The Model

model = Doc2Vec(taggedData, vector_size=5, window=2, min_count=1, workers=4)
fname = get_tmpfile("pdf_corr_doc2vec_model_0.1")

model.save(fname)